In [1]:
#%% import libraries from local directory
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from os.path import relpath
from pybufrkit.decoder import Decoder
from geopandas.tools import sjoin
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points 
import requests
from datetime import datetime, timedelta
from pathlib import Path
from dateutil import rrule
import re
import xml.etree.ElementTree as ET
import xgboost as xgb
import glob

from geopandas.tools import sjoin
 
from shapely.ops import nearest_points
import datetime as datetime
 
 

In [117]:
wor_dir="C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model"

f_path = os.path.join(wor_dir,"IBF-typhoon-model/data/all_predisaster_indicators.csv") 
 
df_predisasters = pd.read_csv(f_path)



def multiplyfun(x, y):
    try:
        value = int((x * y)/100)
        
    except:
        value = np.nan

    return x if value>x else value 

In [118]:
save_dir=os.path.join(wor_dir,"IBF-typhoon-model\\data\\results\\hindcast")
filename_list = sorted(list(Path(save_dir).glob('*.csv')))


dfs=[]
for fname in filename_list:
    leadtime=int(os.path.basename(fname).split('_')[-1].split('.')[0])
    df=pd.read_csv(fname)
    df['leadtime']=leadtime
    dfs.append(df.filter(['Mun_Code', 'storm_id', 'DMG_predicted', 'number_dmg_Build_prediction', 'leadtime','typhoon','year']))

concatenated_df = pd.concat(dfs, ignore_index=True)


IMPACT_DF1 = pd.merge(concatenated_df, df_predisasters[['Housing Units','Mun_Code']],  how='left', left_on='Mun_Code', right_on = 'Mun_Code') 
IMPACT_DF1["number_dmg_Build"] = IMPACT_DF1.apply(lambda x: multiplyfun(x["Housing Units"], x["DMG_predicted"]), axis=1).values

In [111]:
probability_impact=concatenated_df.groupby(['typhoon','storm_id','leadtime']).agg(NUmber_of_affected_municipality=('Mun_Code','count'),
                                    average_ML_Model=('DMG_predicted', 'mean'),
                                    Total_buildings_ML_Model=('number_dmg_Build_prediction', sum)).sort_values(by='Total_buildings_ML_Model',ascending=False).reset_index()

dref_probabilities = {
    "VH1": [100000, 0.45],
    "VH2": [90000, 0.5],
    "H1": [100000, 0.35],
    "H2": [90000, 0.4],
    "H3":  [80000, 0.5],
    "H4": [70000, 0.55],
    "M3":  [80000, 0.6],
    "M2":  [70000, 0.5],
    "M1":  [50000, 0.7],
}

DREF_trigger_list={}

for cyclone in list(set(probability_impact.typhoon.values)):
    df_stormid=probability_impact.query("typhoon==@cyclone")
    if not df.empty:
        for leadtime in list(set(df_stormid.leadtime.values)):
            df_stormid_leadtime=df_stormid.query("leadtime==@leadtime")
            if len(df_stormid_leadtime)>10:
                agg_impact = df_stormid_leadtime["Total_buildings_ML_Model"].values          
                for key, values in dref_probabilities.items():
                    dref_trigger_status = {}      
                    trigger_stat = 100*(sum(1.13*i > values[0] for i in agg_impact) /len(agg_impact))
                    if  (values[1] < (sum(1.13*i > values[0] for i in agg_impact) /len(agg_impact))):
                        trigger_stat_=True
                        thershold=key 
                    else:
                        trigger_stat_=False
                        thershold=key
                    dref_trigger_status['thre_name'] = key
                    dref_trigger_status['thr_value'] = int(trigger_stat)      
                    dref_trigger_status['event'] = cyclone 
                    dref_trigger_status['lead_time'] = int(leadtime) 
                    dref_trigger_status['trigger_stat'] = trigger_stat_
                    index=cyclone+'_'+ str(leadtime) + key
                    
                    DREF_trigger_list[index] = dref_trigger_status
        

Trigger_status=True
impact_path_ = os.path.join(wor_dir,"IBF-typhoon-model/data/results/dref_trigger_table_prob.csv")
pd.DataFrame.from_dict(DREF_trigger_list,orient='index').query('trigger_stat==@Trigger_status')#.to_csv(impact_path_) 

,thre_name,thr_value,event,lead_time,trigger_stat
haiyan_0M1,M1,100,haiyan,0,True
haiyan_12M2,M2,51,haiyan,12,True
haiyan_12M1,M1,100,haiyan,12,True
hagupit_6M1,M1,88,hagupit,6,True
rai_17M1,M1,76,rai,17,True
rai_-6M1,M1,78,rai,-6,True
rai_5M1,M1,82,rai,5,True
